In [ ]:
import requests
import csv
import os
import datetime
import boto3
from collections import defaultdict

# Cliente S3
s3 = boto3.client("s3")

# Diccionario Comunidades
geo_dict = {
    4:"Andalucía", 5:"Aragón", 6:"Cantabria", 7:"Castilla la Mancha",
    8:"Castilla y León", 9:"Cataluña", 10:"País Vasco",
    11:"Principado de Asturias", 8744:"Comunidad de Ceuta",
    8745:"Comunidad de Melilla", 13:"Comunidad de Madrid",
    14:"Comunidad de Navarra", 15:"Comunidad Valenciana",
    16:"Extremadura", 17:"Galicia", 8743:"Islas Baleares",
    8742:"Islas Canarias", 20:"La Rioja", 21:"Región de Murcia"
}

BUCKET_NAME = "proyectohabree"

def lambda_handler(event, context):
    hoy = datetime.date.today()
    start_date = hoy.strftime("%Y-%m-%dT00:00")
    end_date = hoy.strftime("%Y-%m-%dT23:59")
    prefix = f"demanda_diaria/{hoy}/"

    #Solicitud a la API para los datos de hoy
    url = "https://apidatos.ree.es/es/datos/demanda/evolucion"
    params = {
        "start_date": start_date,
        "end_date": end_date,
        "time_trunc": "day",
        "geo_limit": "ccaa"
    }

    resp = requests.get(url, params=params, timeout=20)
    resp.raise_for_status()
    data = resp.json()

    #Agrupar por comunidad
    grupos = defaultdict(list)
    for item in data.get("included", [])[0].get("attributes", {}).get("values", []):
        geo_id_raw = item.get("geo_id")
        if geo_id_raw is None:
            continue  
        try:
            geo_id = int(geo_id_raw)
        except (TypeError, ValueError):
            geo_id = -1 

        comunidad = geo_dict.get(geo_id, "Desconocida")
        row = {
            "datetime": item.get("datetime"),
            "value": item.get("value"),
            "geo_id": geo_id,
            "comunidad": comunidad
        }
        grupos[comunidad].append(row)

    #Crear CSV por comunidad y subir a S3
    for comunidad, filas in grupos.items():
        nombre = comunidad.replace(" ", "_") + ".csv"
        ruta_local = f"/tmp/{nombre}"
        ruta_s3 = prefix + nombre

        with open(ruta_local, "w", newline='', encoding="utf-8") as f_out:
            writer = csv.DictWriter(f_out, fieldnames=["datetime", "value", "geo_id", "comunidad"])
            writer.writeheader()
            writer.writerows(filas)

        s3.upload_file(ruta_local, BUCKET_NAME, ruta_s3)
        print(f"Subido: s3://{BUCKET_NAME}/{ruta_s3}")

    return {
        "statusCode": 200,
        "body": f"Archivos diarios guardados en s3://{BUCKET_NAME}/{prefix}"
    }
